In [1]:
from pyspark.sql import SparkSession
import json

In [2]:
spark = SparkSession.builder.appName("statistics for hotels reviews in Booking").config(
    "spark.some.config.option", "booking").getOrCreate()

In [3]:
spark

In [4]:
data = spark.read.json("/home/yi/006_Paris_tripadvisor.json")

In [5]:
data.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- date: string (nullable = true)
 |-- hotelLocation: string (nullable = true)
 |-- hotelName: string (nullable = true)
 |-- hotelStars: string (nullable = true)
 |-- hotelUrl: string (nullable = true)
 |-- review: string (nullable = true)
 |-- score: double (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- userId: string (nullable = true)



In [6]:
data.count()

349413

In [7]:
# try tp remove duplicate rows in data

data = data.drop_duplicates()

In [8]:
data.count()

349413

In [9]:
statistics = dict()
statistics["totalReviews"] = data.select(data.url).distinct().count()

In [10]:
statistics

{'totalReviews': 198783}

In [21]:
statistics["totalHotels"] = data.select(data.hotelUrl).distinct().count()

In [22]:
statistics["avgReviewNumAHotel"] =  statistics["totalReviews"] / statistics["totalHotels"]

In [23]:
statistics["0 stars"] = 0
statistics["1 stars"] = 0
statistics["2 stars"] = 0
statistics["3 stars"] = 0
statistics["4 stars"] = 0
statistics["5 stars"] = 0
statistics["null stars"] = 0
statistics["eachHotelDetail"]  = list()
statistics["minReviewNumAHotel"] = statistics["totalReviews"]
statistics["maxReviewNumAHotel"] = 0

In [27]:
nullScoreNumAHotel = 0
nullScoreNum = 0
totalScoreAHotel = 0
totalScore = 0
# get statistics for each single hotel
for row in data.groupby(data.hotelUrl).count().collect():
    oneHotel = dict()
    oneHotel["reviewsNum"] = row["count"]
#     print(oneHotel["reviewsNum"])
    if oneHotel["reviewsNum"] > statistics["maxReviewNumAHotel"]:
        statistics["maxReviewNumAHotel"] = oneHotel["reviewsNum"]
    if oneHotel["reviewsNum"] < statistics["minReviewNumAHotel"]:
        statistics["minReviewNumAHotel"] = oneHotel["reviewsNum"]
    oneHotel["hotelUrl"] = row["hotelUrl"]
    try:
        oneHotel["hotelStars"] = data.filter(data.hotelUrl == row["hotelUrl"]).first()["hotelStars"]
    except:
        oneHotel["hotelStars"] = None
    oneHotel["0 stars"] = 0
    oneHotel["1 stars"] = 0
    oneHotel["2 stars"] = 0
    oneHotel["3 stars"] = 0
    oneHotel["4 stars"] = 0
    oneHotel["5 stars"] = 0
    
    for record in data.filter(data.hotelUrl == row["hotelUrl"]).collect():
        if record["score"] == None:
            nullScoreNumAHotel += 1
            nullScoreNum += 1
        else:
#             remove \n newline and convert string into float
            try:
                totalScore += float(record["score"])
                totalScoreAHotel += float(record["score"])
            except:
                nullScoreNumAHotel += 1
                nullScoreNum += 1
                pass
#             apply Department of Tourism (DOT) Star Grading System;
# 5 stars - (0.85, 10]; 4 stars - (0.7, 0.85]; 3 stars - (0.55, 0.7]; 2 stars - (0.4, 0.55]; 1 stars - (0.25, 0.4]
# method: score/10*5; then round.
            oneHotel[str(round(float(record["score"])/10 * 5)) + " stars"] += 1
            statistics[str(round(float(record["score"])/10 * 5)) + " stars"] += 1
    if (oneHotel["reviewsNum"] - nullScoreNumAHotel) == 0:
        oneHotel["avgScore"] = None #means all reviews aren't gaven scores.
    else:
        oneHotel["avgScore"] = totalScoreAHotel / (oneHotel["reviewsNum"] - nullScoreNumAHotel)
    nullScoreNumAHotel = 0
    statistics["eachHotelDetail"].append(oneHotel)
statistics["null stars"] = nullScoreNum
statistics["avgScore"] = totalScore / (statistics["totalReviews"] - nullScoreNum)

In [28]:
statistics

{'0 stars': 0,
 '1 stars': 198,
 '2 stars': 1652,
 '3 stars': 4252,
 '4 stars': 17069,
 '5 stars': 4772,
 'avgReviewNumAHotel': 172.05839416058393,
 'avgScore': 7.690363552691853,
 'eachHotelDetail': [{'0 stars': 0,
   '1 stars': 1,
   '2 stars': 7,
   '3 stars': 25,
   '4 stars': 133,
   '5 stars': 27,
   'avgScore': 7.850259067357507,
   'hotelStars': None,
   'hotelUrl': 'https://www.booking.com/hotel/fi/accome-apartments-parliament.html',
   'reviewsNum': 203},
  {'0 stars': 0,
   '1 stars': 0,
   '2 stars': 4,
   '3 stars': 17,
   '4 stars': 100,
   '5 stars': 48,
   'avgScore': 17.071597633136086,
   'hotelStars': '5 stars',
   'hotelUrl': 'https://www.booking.com/hotel/fi/swhotelkamphelsinki.html',
   'reviewsNum': 203},
  {'0 stars': 0,
   '1 stars': 0,
   '2 stars': 6,
   '3 stars': 6,
   '4 stars': 85,
   '5 stars': 30,
   'avgScore': 30.925984251968522,
   'hotelStars': None,
   'hotelUrl': 'https://www.booking.com/hotel/fi/the-yard-hostel-helsinki.html',
   'reviewsNum': 17

In [32]:
with open("005_statistics_booking_Helsinki_tripadvisor.json", "w") as f:
    json.dump(statistics, f)
    f.close()